In [2]:
!pip install transformers torch torchvision "pillow<10.0" pdf2image pytesseract accelerate

  Using cached pytesseract-0.3.13-py3-none-any.whl.metadata (11 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 56.9 MB/s eta 0:00:00
Using cached pytesseract-0.3.13-py3-none-any.whl (14 kB)
  Attempting uninstall: pillow
    Found existing installation: pillow 11.3.0
    Uninstalling pillow-11.3.0:
      Successfully uninstalled pillow-11.3.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [accelerate]3 [accelerate]


In [8]:
import torch
import json
# AutoProcessor 대신, Processor를 직접 지정하여 문제를 해결합니다.
from transformers import LayoutLMv3Processor, AutoModelForObjectDetection
from PIL import Image
from pdf2image import convert_from_path
import pytesseract
import os

def create_document_pipeline(pdf_path, output_json_path, tesseract_cmd=None):
    """
    PDF 파일을 분석하여 요소, 위치, 텍스트를 JSON으로 반환하는 최종 파이프라인
    """
    if tesseract_cmd:
        pytesseract.pytesseract.tesseract_cmd = tesseract_cmd

    # 1. 모델과 프로세서 준비
    # 실제 모델(fine-tuned weights)은 HYPJUDY의 것을 사용합니다.
    model_name = "tensorlake/layoutlmv3-publaynet"
    # 프로세서(설계도)는 모든 설정 파일이 갖춰진 microsoft의 base 모델에서 가져옵니다.
    processor_name = "microsoft/layoutlmv3-base"

    print(f"✅ 프로세서 로딩: {processor_name}")
    print(f"✅ 모델 웨이트 로딩: {model_name}")

    try:
        # LayoutLMv3Processor를 직접 지정하여 로드합니다.
        processor = LayoutLMv3Processor.from_pretrained(processor_name)
        model = AutoModelForObjectDetection.from_pretrained(model_name)
    except Exception as e:
        print(f"❌ 로딩 실패: {e}")
        print("   인터넷 연결을 확인하시고, 문제가 계속되면 네트워크 방화벽을 의심해볼 수 있습니다.")
        return

    # 2. PDF 변환
    try:
        images = convert_from_path(pdf_path)
    except Exception as e:
        print(f"❌ PDF 변환 오류: Poppler가 제대로 설치되었는지 확인하세요. ({e})")
        return

    final_output = {
        "document_name": os.path.basename(pdf_path),
        "pages": []
    }
    
    element_id_counter = 1

    # 3. 페이지 분석
    for i, image in enumerate(images):
        page_number = i + 1
        print(f"📄 페이지 {page_number} 분석 중...")

        width, height = image.size
        page_data = {
            "page_number": page_number,
            "image_dimensions": {"width": width, "height": height},
            "elements": []
        }

        # 4. 레이아웃 탐지
        # LayoutLMv3Processor는 OCR을 적용하지 않은 이미지만 입력으로 받습니다.
        encoding = processor(images=image, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**encoding)
        
        # 5. 결과 후처리
        target_sizes = torch.tensor([image.size[::-1]])
        results = processor.post_process_object_detection(outputs, threshold=0.7, target_sizes=target_sizes)[0]
        
        for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
            pixel_box = [int(coord) for coord in box.tolist()]
            label_name = model.config.id2label[label.item()]
            
            text_content = None
            # 6. OCR
            if label_name in ["title", "text", "list", "table"]:
                try:
                    cropped_image = image.crop(pixel_box)
                    text_content = pytesseract.image_to_string(cropped_image, lang='kor+eng').strip()
                except Exception:
                    text_content = ""

            page_data["elements"].append({
                "id": element_id_counter,
                "label": label_name,
                "score": round(score.item(), 3),
                "box_2d": pixel_box,
                "text": text_content
            })
            element_id_counter += 1

        final_output["pages"].append(page_data)

    # 7. JSON 저장
    output_dir = os.path.dirname(output_json_path)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    with open(output_json_path, 'w', encoding='utf-8') as f:
        json.dump(final_output, f, ensure_ascii=False, indent=4)

    print(f"\n✨ 분석 완료! 결과가 '{output_json_path}' 파일에 저장되었습니다.")


if __name__ == '__main__':
    # --- 설정 ---
    pdf_file = "./data/GrayAnatomyPDF.pdf"  # ⚠️ 실제 PDF 파일 경로로 변경하세요.
    json_file = "./result/layout/output.json"      # ⚠️ 결과 파일 경로를 지정하세요.
    
    # (Windows 사용자) Tesseract-OCR 설치 경로를 자동으로 찾지 못할 때만 경로를 직접 지정
    tesseract_path = None
    
    # --- 실행 ---
    if os.path.exists(pdf_file):
        create_document_pipeline(pdf_file, json_file, tesseract_path)
    else:
        print(f"❌ 파일을 찾을 수 없습니다: {pdf_file}")

✅ 프로세서 로딩: microsoft/layoutlmv3-base
✅ 모델 웨이트 로딩: tensorlake/layoutlmv3-publaynet
❌ 로딩 실패: Unrecognized configuration class <class 'transformers.models.layoutlmv3.configuration_layoutlmv3.LayoutLMv3Config'> for this kind of AutoModel: AutoModelForObjectDetection.
Model type should be one of ConditionalDetrConfig, DFineConfig, DabDetrConfig, DeformableDetrConfig, DetaConfig, DetrConfig, RTDetrConfig, RTDetrV2Config, TableTransformerConfig, YolosConfig.
   인터넷 연결을 확인하시고, 문제가 계속되면 네트워크 방화벽을 의심해볼 수 있습니다.
